In [35]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
 

In [24]:
#fixing the dataframe 
df=pd.read_csv('vadu-dummy-data.csv',header=1)
df.head()
unique=np.array([df['Class'].unique()])
print(unique)

[['Vata' 'Kapha' 'Pitta']]


In [25]:
#initialised encoder object
encoder=OneHotEncoder(sparse_output=False)
encoded=encoder.fit_transform(df[['Class']])
print(encoded)#this is the numpy 2d array of the classes which are one hot encoded

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1.

In [33]:
#make the numpy array of the encoded classes into data frame
enccolumns=encoder.get_feature_names_out(['Class'])
targetencoded=pd.DataFrame(encoded,columns=enccolumns,index=df.index)
print(targetencoded)
df.head()

     Class_Kapha  Class_Pitta  Class_Vata
0            0.0          0.0         1.0
1            1.0          0.0         0.0
2            0.0          0.0         1.0
3            0.0          0.0         1.0
4            1.0          0.0         0.0
..           ...          ...         ...
126          0.0          0.0         1.0
127          0.0          0.0         1.0
128          0.0          0.0         1.0
129          0.0          0.0         1.0
130          0.0          1.0         0.0

[131 rows x 3 columns]


,SampleID,GENDER,F1,F2,F3,F4,F5,F6,F7,F8,...,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132
0,S1,Male,Weaklydeveloped,Thin/Narrow,Long,Black,Thin/Narrow,DarkBrown,Moderatelydeveloped,Proportionate,...,Non_Irrelevantinbetween,Non_Brittle/Cracked,Non_Loose,Clear,Non_Cracked,Non_Freckles,Non_Marks,Non_Moles,Pimples,Non_Wrinkled
1,S2,Male,Welldeveloped,Broad,Long,DarkBrown,Broad,DarkBrown,Moderatelydeveloped,Proportionate,...,Non_Irrelevantinbetween,Non_Brittle/Cracked,Non_Loose,Clear,Non_Cracked,Non_Freckles,Marks,Non_Moles,Non_Pimples,Non_Wrinkled
2,S3,Male,Weaklydeveloped,Thin/Narrow,Long,LightBrown,Thin/Narrow,LightBrown,Weaklydeveloped,Proportionate,...,Non_Irrelevantinbetween,Non_Brittle/Cracked,Non_Loose,Non_Clear,Cracked,Non_Freckles,Non_Marks,Moles,Pimples,Wrinkled
3,S4,Male,Weaklydeveloped,Thin/Narrow,Long,Dusky,Thin/Narrow,DarkBrown,Weaklydeveloped,Proportionate,...,Irrelevantinbetween,Non_Brittle/Cracked,Non_Loose,Clear,Cracked,Non_Freckles,Non_Marks,Non_Moles,Non_Pimples,Wrinkled
4,S5,Male,Welldeveloped,Broad,Long,Black,Broad,Black,Moderatelydeveloped,Proportionate,...,Non_Irrelevantinbetween,Non_Brittle/Cracked,Non_Loose,Non_Clear,Non_Cracked,Non_Freckles,Marks,Moles,Non_Pimples,Non_Wrinkled


In [27]:
#i need to encode all the rows of data because all are categorical (i am cooked)
#okay i can do labelencoding on all since i am using random forest model


In [38]:
labels=LabelEncoder()
for i in df.columns:
    df[i]=labels.fit_transform(df[i])

In [42]:
df.drop(columns='SampleID')
df=pd.concat()

,GENDER,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F123,F124,F125,F126,F127,F128,F129,F130,F131,F132
0,1,1,2,0,0,2,1,0,1,1,...,1,1,1,0,1,1,1,1,1,0
1,1,2,0,0,1,0,1,0,1,1,...,1,1,1,0,1,1,0,1,0,0
2,1,1,2,0,3,2,3,1,1,0,...,1,1,1,1,0,1,1,0,1,1
3,1,1,2,0,2,2,1,1,1,1,...,0,1,1,0,0,1,1,1,0,1
4,1,2,0,0,0,0,0,0,1,1,...,1,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0,1,2,1,3,2,1,0,1,1,...,0,1,1,1,1,1,1,1,0,1
127,0,1,2,1,3,1,3,0,1,1,...,1,1,1,1,0,0,0,0,0,0
128,0,1,2,2,1,2,0,1,1,1,...,1,1,1,1,1,1,0,0,0,0
129,0,1,1,1,3,1,3,0,1,0,...,1,1,1,0,1,0,1,0,0,1


In [46]:
list(labels.inverse_transform(df['GENDER']))#like this i can check the labels too even when they are encoded

['Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Non_Wrinkled',
 'Wrinkled',
 'Wri